In [ ]:
!pip install tensorflow
import tensorflow as tf
import tensorflow_datasets as tfds
import datetime

In [ ]:
dataset_name = "eurosat/rgb"
dataset, info = tfds.load(dataset_name, split="train", as_supervised=True, with_info=True)

In [ ]:
import matplotlib.pyplot as plt

def show_12th_per_class(dataset, num_classes, class_names):
    plt.figure(figsize=(15, 10))
    samples = {i: None for i in range(num_classes)}
    counts = {i: 0 for i in range(num_classes)}

    for image, label in dataset:
        label_val = label.numpy()
        counts[label_val] += 1

        if counts[label_val] == 12:
            samples[label_val] = (image, label)

        if all(v is not None for v in samples.values()):
            break

    for i, (image, label) in enumerate(samples.values()):
        plt.subplot(3, (num_classes + 2) // 3, i + 1)
        plt.imshow(image.numpy())
        plt.title(class_names[label.numpy()], fontsize=16)
        plt.xticks([])
        plt.yticks([])
        plt.axis("off")

    plt.tight_layout()
    plt.show()


show_12th_per_class(dataset, len(class_names), class_names)

In [ ]:
print(info)

In [ ]:
class_names = info.features["label"].names
print("EuroSAT Class Names:")
for i, name in enumerate(class_names):
    print(f"{i}: {name}")

In [ ]:
image, label = next(iter(dataset))

tensor_shape = tf.shape(image)

print(f"Shape of image: {tensor_shape}")

In [ ]:
train_size = len(dataset)

print(f"Number of samples in the train dataset: {train_size}")

In [ ]:
!pip install scikit-learn

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
total_size = tf.data.experimental.cardinality(dataset).numpy()
train_size = int(0.7 * total_size)  # 70% for training
val_size = int(0.1 * total_size)    # 10% for validation
test_size = total_size - train_size - val_size  # 20% for testing

In [ ]:
BUFFER_SIZE = 27000
dataset = dataset.shuffle(BUFFER_SIZE, reshuffle_each_iteration=False)

In [ ]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)
test_dataset = dataset.skip(train_size + val_size)

In [ ]:
BATCH_SIZE = 16

train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom

data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.2),
    RandomZoom(0.2),
])

In [ ]:
test_dataset = test_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))
train_dataset = test_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))
val_dataset = val_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
base_model = tf.keras.applications.EfficientNetV2S(
    weights="imagenet",
    include_top=False,
    input_shape=(64, 64, 3)
)

In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define Early Stopping Callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,          # Stop training if val_loss doesn't improve for 5 epochs
    restore_best_weights=True  # Restore best model weights after stopping
)

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=20,callbacks=[early_stopping])

In [ ]:
model.save('/content/drive/My Drive/modelEFFICIENTNETV2S.h5')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16.53, 11.69))
ax1.plot(history.history['accuracy'])
ax1.plot(history.history['val_accuracy'])
ax1.set_xlabel('epoch')
ax1.set_ylabel('accuracy')
ax1.set_title('Accuracy over epoch')
ax1.legend(['Train', 'Test'], loc='upper left')

ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
ax2.set_xlabel('epoch')
ax2.set_ylabel('loss')
ax2.set_title('Loss over epoch')
ax2.legend(['Train', 'Test'], loc="upper right")